In [1]:
import csv
import json
import os
import numpy as np
import nibabel as nib
import pandas as pd
import suspect
import glob
import ants
from multiprocessing import Pool

In [2]:
# Get Subject IDs

In [3]:
sublist = [i.split('/')[-1].split('.')[-4] for i in glob.glob('/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subject_space_Voxels/*.mUNIbrain_DENskull_SPMmasked_mrs_voxel_center.nii.gz')]

In [4]:
sublist[0]

'NeuroMet001'

In [ ]:
# Coregister the Center of the MRS Voxel to MNI

In [ ]:
def apply_mni_trans(sub_id):
    dest_file = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/MNI_space_voxels/{}_MRS_Voxel_center_MNIwarped.nii.gz'.format(sub_id)
    if not os.path.isfile(dest_file):
        try:
            mov_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subject_space_Voxels/{sub_id}.mUNIbrain_DENskull_SPMmasked_mrs_voxel_center.nii.gz'
            mni_std = '/usr/share/fsl/data/standard/MNI152_T1_0.5mm.nii.gz'
            ants_mni_std = ants.image_read(mni_std)
            transform1_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg0GenericAffine.mat'
            transform2_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg1Warp.nii.gz'
            print(sub_id)
            transform1 = transform1_mask.format(sub_id=sub_id)
            transform2 = transform2_mask.format(sub_id=sub_id)
            moving = mov_mask.format(sub_id=sub_id)
            ants_mov = ants.image_read(moving)
            res = ants.apply_transforms(fixed=ants_mni_std,
                             moving=ants_mov,
                             transformlist = [transform2, transform1])
            ants.image_write(res, dest_file)
            return '{sub} OK'.format(sub=sub_id)
        except:
            return '{sub} FAILED'.format(sub=sub_id)

In [ ]:
with Pool(4) as p:
    print(p.map(apply_mni_trans, sublist))

In [ ]:
# Calculate distance between MRS Voxel in MNI and L/R-PCC

In [9]:
from scipy.spatial import distance
def pcc_dist(sub_id):
    print(sub_id)
    # Return dict {subid:{lpccdist:m,rccdist:n}}
    img = nib.load('/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/MNI_Space_Voxels/{sub}_MRS_Voxel_center_MNIwarped.nii.gz'.format(sub=sub_id))
    _, x, y, z, _, _, _, _ = img.header['pixdim']
    img_data = img.get_fdata()
    voxel_vol = x*y*z
    lpcc = (101,67,85)
    rpcc = (77,67,92)
    coords = (np.where(img_data==img_data.max())[0][0],
              np.where(img_data==img_data.max())[1][0],
              np.where(img_data==img_data.max())[2][0])
    print(coords)
    return {sub_id : {'dist_rpcc_voxel':distance.euclidean(coords,rpcc),
                     'dist_lpcc_voxel': distance.euclidean(coords,lpcc),
                     'dist_rpcc_mm':distance.euclidean(coords,rpcc)*voxel_vol,
                     'dist_lpcc_mm': distance.euclidean(coords,lpcc)*voxel_vol,}}

In [10]:
with Pool(4) as p2:
    distances = p2.map(pcc_dist, sublist)

NeuroMet013
NeuroMet019
NeuroMet001
NeuroMet007
(175, 159, 203)
NeuroMet014
(170, 158, 210)
NeuroMet020
(173, 157, 209)
NeuroMet008
(180, 154, 207)
NeuroMet002
(177, 144, 199)
NeuroMet015
(178, 146, 207)
Neuromet021
(176, 173, 210)
NeuroMet003
(181, 153, 201)
NeuroMet009
(178, 153, 196)
NeuroMet016
(179, 156, 208)
NeuroMet004
(177, 159, 212)
NeuroMet010
(180, 159, 206)
NeuroMet022
(179, 162, 207)
NeuroMet017
(182, 147, 205)
(181, 151, 202)
NeuroMet011
Neuromet005
(179, 149, 201)
NeuroMet023
(182, 153, 194)
NeuroMet018
(177, 155, 212)
NeuroMet012
(177, 178, 211)
NeuroMet006
(177, 155, 203)
NeuroMet024
(176, 155, 206)
NeuroMet025
(179, 160, 208)
NeuroMet031
(175, 157, 207)
NeuroMet037
(181, 159, 196)
NeuroMet044
(183, 157, 203)
NeuroMet026
(180, 152, 216)
(182, 160, 213)
NeuroMet038
NeuroMet032
(181, 157, 224)
NeuroMet045
(179, 155, 203)
NeuroMet027
(175, 160, 198)
NeuroMet033
(178, 150, 199)
NeuroMet046
(181, 153, 204)
NeuroMet039
(175, 150, 209)
NeuroMet028
(177, 151, 208)
NeuroMet034


In [11]:
dist_dict = distances[0]
for i in distances:
    dist_dict.update(i)

In [12]:
pd.DataFrame(dist_dict).transpose()

,dist_rpcc_voxel,dist_lpcc_voxel,dist_rpcc_mm,dist_lpcc_mm
NeuroMet001,177.208916,169.393034,22.151115,21.174129
NeuroMet002,186.978608,180.238731,23.372326,22.529841
NeuroMet003,178.272264,170.686848,22.284033,21.335856
NeuroMet004,173.124233,164.757397,21.640529,20.594675
Neuromet005,191.002618,184.317661,23.875327,23.039708
...,...,...,...,...
NeuroMET086,172.594901,164.748293,21.574363,20.593537
NeuroMET087,180.986187,173.484870,22.623273,21.685609
NeuroMET088,171.376778,163.490061,21.422097,20.436258
NeuroMET089,168.798697,160.480528,21.099837,20.060066


In [13]:
pd.DataFrame(dist_dict).transpose().to_csv('distances_pcc_mrs.csv')